In [1]:
folder_name = 'unzpd' # папка с исходными xls-файлами

dst_file_name = 'ri_collection.xlsx' # файл, в который собираем данные из xls-файлов

cols=['file_guid','sheet_name',
      'rek','tr_date','tr_num','tr_source',
      'region','org','inn','kpp',
      'ter','tar','ski','det','potr','scheme','nositel','source',
      'tar_value','date_begin','date_end'] # список полей файла, в который собираем данные

sheets=['Форма 4.2.1 | Т-ТЭ | ТСО',
        'Форма 4.2.1 | Т-ТЭ | потр',
        'Форма 4.2.1 | Т-ТЭ | предел',
        'Форма 4.2.1 | Т-ТЭ | индикат',
        'Форма 4.2.1 | Т-ТЭ | >=25МВт',
        'Форма 4.2.2 | Т-передача ТЭ',
        'Форма 4.2.2 | Т-передача ТН',
        'Форма 4.2.2 | Т-ТН'] # список листов, из которых собираем данные

In [2]:
%%time

import pandas as pd
import os
from pyxlsb import open_workbook
import sys

df = pd.DataFrame(columns=cols)

files = os.listdir(folder_name)
#collected_data = pd.read_excel(dst_file_name, usecols = cols)
collected_data = pd.DataFrame(columns=cols)


for filename in files:
    if filename.endswith('.xlsb'):
        if ( len(collected_data[collected_data['file_guid'].str.contains(filename[0:36])]) == 0 ) :
            try:
                wb = open_workbook(folder_name+'/'+filename)
                
                tar_cnt = 0

                ws = wb.get_sheet('Титульный')

                rek = 'Не задано' #Наименование органа регулирования, принявшего решение об утверждении тарифов
                tr_date = 'Не задано' #Дата документа об утверждении тарифов
                tr_num = 'Не задано' #Номер документа об утверждении тарифов
                tr_source = 'Не задано' #Источник официального опубликования решения
                region = 'Не задано' #Субъект РФ
                org = 'Не задано' #Наименование организации
                inn = 'Не задано' #ИНН
                kpp = 'Не задано' #КПП

                for row in ws.rows():
                    if (row[4].v == 'Наименование органа регулирования, принявшего решение об утверждении тарифов'): rek = row[5].v
                    if (row[4].v == 'Дата документа об утверждении тарифов'): tr_date = row[5].v
                    if (row[4].v == 'Номер документа об утверждении тарифов'): tr_num = row[5].v
                    if (row[4].v == 'Источник официального опубликования решения'): tr_source = row[5].v
                    if (row[4].v == 'Субъект РФ'): region = row[5].v
                    if (row[4].v == 'Наименование организации'): org = row[5].v
                    if (row[4].v == 'ИНН'): inn = row[5].v
                    if (row[4].v == 'КПП'): kpp = row[5].v

                for sheetname in sheets:
                    ws = wb.get_sheet(sheetname)

                    tar = 'Не задано' #Наименование тарифа
                    ter = 'Не задано' #Территория действия тарифа
                    ski = 'Не задано' #Наименование централизованной системы холодного водоснабжения
                    det = 'Не задано' #Наименование признака дифференциации
                    potr = 'Не задано' #Группа потребителей
                    scheme = 'Не задано' #Схема подключения
                    source = 'Не задано' #Источник тепловой энергии
                    nositel = 'Не задано' #Теплоноситель
                    
                    for row in ws.rows():
                        if (row[0].r >= 12):
                            if (row[12].v == 'Наименование тарифа'): tar = row[14].v
                            if (row[12].v == 'Территория действия тарифа'): ter = row[14].v
                            if (row[12].v == 'Наименование системы теплоснабжения '): ski = row[14].v
                            if (row[12].v == 'Группа потребителей'): potr = row[14].v
                            if (row[12].v == 'Схема подключения теплопотребляющей установки к коллектору источника тепловой энергии'): scheme = row[14].v
                            if (row[12].v == 'Источник тепловой энергии'): source = row[14].v

                            for i in range(10):
                                if (len(row) >= 19+i*7):
                                    if ( (row[14+i*7].v != None)&(row[17+i*7].v != None)&(row[19+i*7].v != None) ):
                                        df = df.append({'file_guid' : filename[0:36], 'sheet_name': sheetname,
                                                        'rek': rek, 'tr_date': tr_date, 'tr_num': tr_num, 'tr_source': tr_source,
                                                        'region': region, 'org': org, 'inn': inn, 'kpp': kpp,
                                                        'ter': ter, 'tar': tar, 'ski': ski, 'scheme': scheme, 'source':source, 'potr': potr, 'nositel': row[12].v,
                                                        'tar_value': row[14+i*7].v, 'date_begin': row[17+i*7].v, 'date_end': row[19+i*7].v
                                                       }, ignore_index=True)
                                        tar_cnt = tar_cnt+1

                # это добавляем, чтобы при отсутствии данных по тарифам была какая-то запись о файле
                if (tar_cnt == 0):
                    df = df.append({'file_guid' : filename[0:36],
                                    'rek': rek, 'tr_date': tr_date, 'tr_num': tr_num, 'tr_source': tr_source,
                                    'region': region, 'org': org, 'inn': inn, 'kpp': kpp}, ignore_index=True)
            except:
                print("Error with file: "+filename, sys.exc_info()[0])


collected_data = collected_data.append(df)
collected_data = collected_data.drop_duplicates(subset = cols)

print(collected_data.info())
collected_data.to_excel(dst_file_name)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 0 to 159
Data columns (total 21 columns):
file_guid     99 non-null object
sheet_name    99 non-null object
rek           99 non-null object
tr_date       99 non-null object
tr_num        99 non-null object
tr_source     46 non-null object
region        99 non-null object
org           99 non-null object
inn           99 non-null object
kpp           99 non-null object
ter           99 non-null object
tar           59 non-null object
ski           99 non-null object
det           0 non-null float64
potr          99 non-null object
scheme        99 non-null object
nositel       99 non-null object
source        99 non-null object
tar_value     99 non-null float64
date_begin    99 non-null object
date_end      99 non-null object
dtypes: float64(2), object(19)
memory usage: 17.0+ KB
None
CPU times: user 2.49 s, sys: 152 ms, total: 2.64 s
Wall time: 2.7 s
